In [66]:
import pandas as pd

In [67]:
# setup 
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [68]:
def getInputPath():
  return ''

In [69]:
def getOutputPath():
  return ''

In [70]:
def getBaseDataPath():
  return ''

In [71]:
metro = pd.read_csv(getInputPath() + 'estaciones_metro.csv')
metro['TIPOTRANSPORTE'] = 'Metro'


In [72]:
autobus = pd.read_csv(getInputPath() + 'estaciones_autobus_emt.csv')
autobus['TIPOTRANSPORTE'] = 'Autobus'

In [73]:
transporte = pd.concat([metro, autobus])
transporte.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4988 entries, 0 to 4698
Data columns (total 57 columns):
ACONDICIONAMIENTOVEHICULOS        4988 non-null int64
ACONDICIONAMIENTOVIAJEROS         4988 non-null int64
BARRIO                            4643 non-null float64
CALIFICADORPORTAL                 288 non-null object
CARRETERA                         0 non-null float64
CODIGOCTMESTACIONREDMETRO         290 non-null float64
CODIGOEMPRESA                     4988 non-null int64
CODIGOENTIDAD                     4988 non-null int64
CODIGOESTACION                    4988 non-null int64
CODIGOINTERCAMBIADOR              1381 non-null float64
CODIGOMUNICIPIO                   4988 non-null int64
CODIGONUCLEO                      3192 non-null float64
CODIGOPOSTAL                      4987 non-null float64
CODIGOPROVINCIA                   4988 non-null int64
CODIGOVIA                         4988 non-null object
CORONA123                         4986 non-null float64
CORONATARIFARIA   

C:\Users\Carolina\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [74]:
transporte = transporte.loc[:,['BARRIO', 'CODIGOPOSTAL', 'DENOMINACION', 'DISTRITO', 'LATITUD', 'LONGITUD', 'TIPOTRANSPORTE']]

In [75]:
nas_dictionary = {"columns":[], "nas_count":[]}
for column in transporte.columns:
    nas_dictionary["columns"].append(column)
    nas_dictionary["nas_count"].append(len(transporte[column])-transporte[column].count())
nas_dictionary
nas_dataframe = pd.DataFrame(nas_dictionary)
print('Total filas:',len(transporte))
nas_dataframe

Total filas: 4988


,columns,nas_count
0,BARRIO,345
1,CODIGOPOSTAL,1
2,DENOMINACION,0
3,DISTRITO,0
4,LATITUD,0
5,LONGITUD,0
6,TIPOTRANSPORTE,0


In [76]:
!pip install shapely

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [77]:
import json
import numpy as np
from shapely.geometry import shape, Point

with open(getBaseDataPath() + 'MADRID.geojson') as f:
    js = json.load(f)
    
for index, row in transporte.iterrows():
    point = Point(row['LONGITUD'], row['LATITUD'])
    encontrado = False
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            transporte.loc[index, 'CODIGOPOSTAL'] = float(feature['properties']['COD_POSTAL'])
            encontrado = True
            break
    if not encontrado:
      transporte.loc[index, 'CODIGOPOSTAL'] = np.nan

In [78]:
import json
import numpy as np
from shapely.geometry import shape, Point

with open(getBaseDataPath() + 'Barrios.geojson') as f:
    js = json.load(f)
for index, row in transporte.iterrows():
    point = Point(row['LONGITUD'], row['LATITUD'])
    encontrado = False
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            transporte.loc[index, 'BARRIO']=float(feature['properties']['CODBAR'])
            transporte.loc[index, 'DISTRITO']=float(feature['properties']['CODDIS'])
            transporte.loc[index, 'BARRIO_NOMBRE']=str(feature['properties']['NOMBRE'])
            transporte.loc[index, 'DISTRITO_NOMBRE']=str(feature['properties']['NOMDIS'])
            encontrado = True
    if not encontrado:
        transporte.loc[index, 'BARRIO'] = np.nan
        transporte.loc[index, 'DISTRITO'] =np.nan
        transporte.loc[index, 'BARRIO_NOMBRE'] = np.nan
        transporte.loc[index, 'DISTRITO_NOMBRE'] = np.nan

In [79]:
transporte.head()

,BARRIO,CODIGOPOSTAL,DENOMINACION,DISTRITO,LATITUD,LONGITUD,TIPOTRANSPORTE,BARRIO_NOMBRE,DISTRITO_NOMBRE
0,94.0,28039.0,NOVICIADO,9.0,40.424842,-3.707419,Metro,Valdezarza,Moncloa - Aravaca
1,93.0,28040.0,ESTRELLA,9.0,40.411472,-3.661785,Metro,Ciudad Universitaria,Moncloa - Aravaca
2,93.0,28040.0,RIOS ROSAS,9.0,40.441975,-3.701328,Metro,Ciudad Universitaria,Moncloa - Aravaca
3,61.0,28039.0,CAMPAMENTO,6.0,40.394808,-3.768128,Metro,Bellas Vistas,TetuÃ¡n
4,61.0,28039.0,USERA,6.0,40.387097,-3.706896,Metro,Bellas Vistas,TetuÃ¡n


In [80]:
transporte.loc[transporte.BARRIO.isna()]

,BARRIO,CODIGOPOSTAL,DENOMINACION,DISTRITO,LATITUD,LONGITUD,TIPOTRANSPORTE,BARRIO_NOMBRE,DISTRITO_NOMBRE
423,NaN,28223.0,CTRA. HUMERA A POZUELO,NaN,40.428256,-3.788409,Autobus,NaN,NaN
616,NaN,28223.0,CTRA.HUMERA A POZUELO (HUMERA),NaN,40.428397,-3.788632,Autobus,NaN,NaN
617,NaN,28223.0,CAMPUS SOMOSAGUAS-APARCAMIENTO,NaN,40.431983,-3.791976,Autobus,NaN,NaN
4385,NaN,28223.0,FACULTADES DE POLITICAS Y SOCIOLOGIA,NaN,40.430914,-3.790847,Autobus,NaN,NaN


In [81]:
transporte = transporte.dropna()
len(transporte)

4984

In [84]:
!pip install folium

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [82]:
transporte.loc[:, 'DENOMINACION'] = transporte.DENOMINACION.replace("\'", "", regex=True)

In [83]:
transporte[1230:1250]

,BARRIO,CODIGOPOSTAL,DENOMINACION,DISTRITO,LATITUD,LONGITUD,TIPOTRANSPORTE,BARRIO_NOMBRE,DISTRITO_NOMBRE
944,56.0,28036.0,AV.BURGOS Nº30,5.0,40.479608,-3.675785,Autobus,Castilla,ChamartÃ­n
945,56.0,28036.0,BAMBU-YUCA,5.0,40.476632,-3.676298,Autobus,Castilla,ChamartÃ­n
946,56.0,28036.0,AVDA.BURGOS-AVDA.PIO XII,5.0,40.472220,-3.675947,Autobus,Castilla,ChamartÃ­n
947,56.0,28046.0,Pº. CASTELLANA-HOSPITAL LA PAZ,5.0,40.480435,-3.685243,Autobus,Castilla,ChamartÃ­n
948,166.0,28050.0,AVDA.MANOTERAS-AVDA.BURGOS,16.0,40.488945,-3.672114,Autobus,Valdefuentes,Hortaleza
949,166.0,28050.0,OÑA-AV.MANOTERAS,16.0,40.488814,-3.669788,Autobus,Valdefuentes,Hortaleza
950,166.0,28050.0,OÑA Nº111,16.0,40.489479,-3.662279,Autobus,Valdefuentes,Hortaleza
951,171.0,28021.0,DR.MARTIN AREVALO-Pº ALBERTO PALACIOS,17.0,40.346304,-3.708259,Autobus,"Villaverde Alto, Casco HistÃ³rico de Villaverde",Villaverde
952,174.0,28021.0,CTRA.VILLAVERDE A VALLECAS-VILLAFUERTE,17.0,40.352670,-3.690992,Autobus,Los Rosales,Villaverde
953,174.0,28021.0,CTRA. VILLAVERDE A VALLECAS-AV.ROSALES,17.0,40.360134,-3.682659,Autobus,Los Rosales,Villaverde


In [94]:

import folium
map_osm = folium.Map(location=[40.44, -3.69],
                     tiles='CartoDB Positron',
                     # tiles='Mapbox Bright',
                     zoom_start=14)

for index, row in transporte.sample(n=2000).iterrows():
  if row['TIPOTRANSPORTE'] == 'Metro':
    folium.Marker(location = [row['LATITUD'], row['LONGITUD']],
                  #popup=row['DENOMINACION'],
                  icon=folium.Icon(color='red', icon='subway', prefix='fa')).add_to(map_osm)
  else:
    folium.Marker(location = [row['LATITUD'], row['LONGITUD']],
                  #popup=row['DENOMINACION'],
                  icon=folium.Icon(color='blue', icon='bus', prefix='fa')).add_to(map_osm)
    
display(map_osm)